# Load dataset

Data format:

|id|word_seq|tag_seq|
|:--|:--|:--|
|index of the sentence|tokenized words|corresponding NER tags|
|0|`["protection", "calves", ...]`|`["O", "LIVESTOCK", ...]`|
|1|`["prevent", "diarrhea",...]` |`["O", "DISEASE_OR_SYNDROME", ...]`|
|...|...|...|



There are 64 categories of NER tags (plus 1 padding token).

The ground-truth tags are provided for the training and testing set, while being omitted in the testing set.

In [2]:
import keras
from keras.utils import to_categorical
import numpy as np
import os
import pickle as pkl

train_dict = pkl.load(open("data/train.pkl", "rb"))
val_dict = pkl.load(open("data/val.pkl", "rb"))
test_dict = pkl.load(open("data/test.pkl", "rb"))
print("keys in train_dict:", train_dict.keys())
print("keys in val_dict:", val_dict.keys())
print("keys in test_dict:", test_dict.keys())

keys in train_dict: dict_keys(['id', 'word_seq', 'tag_seq'])
keys in val_dict: dict_keys(['id', 'word_seq', 'tag_seq'])
keys in test_dict: dict_keys(['id', 'word_seq'])


In [ ]:
# an entry of the dataset
print("index:", train_dict["id"][0])
print(*zip(train_dict["word_seq"][0], train_dict["tag_seq"][0]))

In [2]:
# all the NER tags:
from itertools import chain
print("count of the NER tags:", len(set(chain(*train_dict["tag_seq"]))))
print("all the NER tags:", set(chain(*train_dict["tag_seq"])))

count of the NER tags: 65
all the NER tags: {'GPE', 'LANGUAGE', 'DAILY_OR_RECREATIONAL_ACTIVITY', 'PERSON', 'LABORATORY_OR_TEST_RESULT', 'ORG', 'MONEY', 'TIME', 'PHYSICAL_SCIENCE', 'CELL_FUNCTION', 'BODY_SUBSTANCE', 'INDIVIDUAL_BEHAVIOR', 'ANATOMICAL_STRUCTURE', 'MATERIAL', 'LIVESTOCK', 'BACTERIUM', 'EUKARYOTE', 'GROUP_ATTRIBUTE', 'VIRAL_PROTEIN', 'NORP', 'CHEMICAL', 'FAC', 'DISEASE_OR_SYNDROME', 'LABORATORY_PROCEDURE', 'EXPERIMENTAL_MODEL_OF_DISEASE', 'VIRUS', 'SOCIAL_BEHAVIOR', 'LAW', 'INJURY_OR_POISONING', '_t_pad_', 'CORONAVIRUS', 'O', 'PRODUCT', 'QUANTITY', 'ARCHAEON', 'FOOD', 'CELL', 'ORGANISM', 'IMMUNE_RESPONSE', 'GROUP', 'ORDINAL', 'CELL_OR_MOLECULAR_DYSFUNCTION', 'LOC', 'WORK_OF_ART', 'GOVERNMENTAL_OR_REGULATORY_ACTIVITY', 'EDUCATIONAL_ACTIVITY', 'DATE', 'WILDLIFE', 'SUBSTRATE', 'CARDINAL', 'EVOLUTION', 'EVENT', 'THERAPEUTIC_OR_PREVENTIVE_PROCEDURE', 'MACHINE_ACTIVITY', 'CELL_COMPONENT', 'SIGN_OR_SYMPTOM', 'MOLECULAR_FUNCTION', 'GENE_OR_GENOME', 'BODY_PART_ORGAN_OR_ORGAN_COMPO

# Prepare the data for training

In [3]:
# prepare word vocab and tag vocab

vocab_dict = {'_unk_': 0, '_w_pad_': 1}

for doc in train_dict['word_seq']:
    for word in doc:
        if(word not in vocab_dict):
            vocab_dict[word] = len(vocab_dict)

tag_dict = {'_t_pad_': 0} # add a padding token

for tag_seq in train_dict['tag_seq']:
    for tag in tag_seq:
        if(tag not in tag_dict):
            tag_dict[tag] = len(tag_dict)
word2idx = vocab_dict
idx2word = {v:k for k,v in word2idx.items()}
tag2idx = tag_dict
idx2tag = {v:k for k,v in tag2idx.items()}            

print("size of word vocab:", len(vocab_dict), "size of tag_dict:", len(tag_dict))

size of word vocab: 82275 size of tag_dict: 65


In [4]:
# The maximum length of a sentence is set to 128
max_sent_length = 128

train_tokens = np.array([[word2idx[w] for w in doc] for doc in train_dict['word_seq']])
val_tokens = np.array([[word2idx.get(w, 0) for w in doc] for doc in val_dict['word_seq']])
test_tokens = np.array([[word2idx.get(w, 0) for w in doc] for doc in test_dict['word_seq']])


train_tags = [[tag2idx[t] for t in t_seq] for t_seq in train_dict['tag_seq']]
train_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) for t_seq in train_tags])

val_tags = [[tag2idx[t] for t in t_seq] for t_seq in val_dict['tag_seq']]
val_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) for t_seq in val_tags])

# we don't have test tags

In [ ]:
print("training size:", train_tokens.shape, "tag size:", train_tags.shape)
print("validating size:", val_tokens.shape, "tag size:", val_tags.shape)
print(train_dict['word_seq'][:2])
print(np.array(train_dict['word_seq']).shape)
print(train_tokens[:2])

In [17]:
# an example of training instance and training tags.
print(train_tokens[0,:10], np.argmax(train_tags[0, :10, :], axis=1))
print(train_dict['word_seq'][0][:10], train_dict['tag_seq'][0][:10])

[ 2  3  4  5  6  7  8  9 10 11] [1 1 2 1 1 3 3 1 4 4]
['Protection', 'of', 'calves', 'against', 'fatal', 'enteric', 'colibacillosis', 'by', 'orally', 'administered'] ['O', 'O', 'LIVESTOCK', 'O', 'O', 'DISEASE_OR_SYNDROME', 'DISEASE_OR_SYNDROME', 'O', 'GENE_OR_GENOME', 'GENE_OR_GENOME']


# Two simple models and codes for evaluation

1. Predict all the tags as "O".
2. Random guess

You could use the `calc_accuracy` function to evaluate the accuracy of your predictions.

In [6]:
# Provided function to test accuracy
# You could check the validation accuracy to select the best of your models
def calc_accuracy(preds, tags, padding_id="_t_pad_"):
    """
        Input:
            preds (np.narray): (num_data, length_sentence)
            tags  (np.narray): (num_data, length_sentence)
        Output:
            Proportion of correct prediction. The padding tokens are filtered out.
    """
    preds_flatten = preds.flatten()
    tags_flatten = tags.flatten()
    non_padding_idx = np.where(tags_flatten!=padding_id)[0]
    
    return sum(preds_flatten[non_padding_idx]==tags_flatten[non_padding_idx])/len(non_padding_idx)

In [6]:
# Check accuracy on the training set
train_tags_by_idx = np.argmax(train_tags, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])

print(calc_accuracy(train_labels, train_labels))

# Predict all labels as "O"
# np.ones will create a matrix that contain all ones and idx2tag will change 1 to O 

baseline1_train_preds = np.array([[idx2tag[p] for p in preds] for preds in np.ones(train_labels.shape)])
print(baseline1_train_preds.shape)
print( train_labels.shape)
print("baseline 1, make all predictions as 1. Acc:", 
      calc_accuracy(baseline1_train_preds, 
                    train_labels))

# Randomly guess labels.
baseline2_train_preds = np.array([[idx2tag[p] for p in preds] for preds in np.random.randint(1, len(tag_dict), train_labels.shape)]) 
print("baseline 2, Random guess. Acc:", 
      calc_accuracy(baseline2_train_preds,
                    train_labels))



1.0
(23600, 128)
(23600, 128)
baseline 1, make all predictions as 1. Acc: 0.7562260387120905
baseline 2, Random guess. Acc: 0.015601057744979706


In [4]:
train_x = train_dict['word_seq'] 
train_y = train_dict ['tag_seq'] 
valid_x = val_dict ['word_seq']
valid_y = val_dict[ 'tag_seq'] 
test_x = test_dict['word_seq']
print(np.array(train_x).shape , np.array(train_y).shape, np.array(valid_x).shape, np.array(valid_y).shape)




(23600, 128) (23600, 128) (2950, 128) (2950, 128)


In [1]:


import kashgari
from kashgari.tasks.labeling import BiGRU_Model, BiGRU_CRF_Model, BiLSTM_CRF_Model,  BiLSTM_Model ,CNN_LSTM_Model 
# from kashgari.embeddings import TransformerEmbedding
import os 

def getModel(name ) : 
    return {
        'BiGRU_Model' : BiGRU_Model () , 
        'BiGRU_CRF_Model' : BiGRU_CRF_Model() , 
        'BiLSTM_CRF_Model' : BiLSTM_CRF_Model ( ) , 
        'BiLSTM_Model' : BiLSTM_Model() , 
        'CNN_LSTM_Model' : CNN_LSTM_Model() , 
    }[name]

def getModelClass(name): 
    return {
        'BiGRU_Model' : BiGRU_Model , 
        'BiGRU_CRF_Model' : BiGRU_CRF_Model, 
        'BiLSTM_CRF_Model' : BiLSTM_CRF_Model, 
        'BiLSTM_Model' : BiLSTM_Model , 
        'CNN_LSTM_Model' : CNN_LSTM_Model , 
    }[name]

models = ['BiGRU_Model' , 'BiGRU_CRF_Model', 'BiLSTM_CRF_Model' , 'BiLSTM_Model' ,'CNN_LSTM_Model' ]




In [10]:
f = {
    'something' : {'a' : 1},
    'something2' : {'a' : 1}, 
    'soemthing3' : {'a' : 1} , 
} 

for item in f : 
    print(f[item]['a'])

d = dict() 
d['a'] =3 
d['b'] = 4 
d['b']

1
1
1


4

In [7]:
import pandas as pd 
import numpy as np 
# df = pd.DataFrame(columns = ["model" , "embedding" ] )
# df2 = none
# d = {'col1': [1], 'col2': [3]}
# df = pd.DataFrame(data =d )
df = pd.DataFrame(columns = ['model' , 'epoch' , 'batch_size' , 'max_f1_score' , 'which_maxf1' , 'support_maxf1' , 'min_f1_score' , 'which_minf1' , 'support_minf1' , 'macro_avg_f1' 'train_acc' , 'val_acc'] )
print(df)

for e in [5 , 10  ] : 
    for b in [64, 128 ] : 
        for m in models : 
            # for em in embeddings : 
                model_folder = f'{m}_{b}_{e}'
                if os.path.isdir(model_folder) : 
                    model = getModelClass(m) .load_model(model_folder)
                else : 
                    model = getModel(m)
                    model.fit(train_x, train_y , valid_x, valid_y , batch_size=b, epochs=e)
                    model.save(model_folder)
                train_preds = model.predict(train_x)
                val_preds = model.predict(valid_x)
                train_report = model.evaluate(train_x , train_y )
                val_report = model.evaluate(valid_x , valid_y )
                train_acc = calc_accuracy(np.array(train_preds) , np.array(train_y) )  ; 
                val_acc = calc_accuracy(np.array(val_preds) , np.array(valid_y ) )

                print() 
                print(f"{model_folder} train preds : " , train_acc) 
                print(f"{model_folder} valid pred : " , val_acc) 
                print() 
                # df = pd.DataFrame(columns = ['model' , 'epoch' , 'batch_size' , 'max_f1_score' , 'which_maxf1' , 'support_maxf1' , 'min_f1_score' , 'which_minf1' , 'support_minf1' , 'train_acc' , 'val_acc'] )

                max_f1_score = 0  
                which_max_f1 = "" 
                support_max_f1 = 0  
                min_f1_score = 1
                which_min_f1 = '' 
                support_max_f1 = 0 
                for item in train_report['detail'] :
                    temp = train_report['detail'][item] 
                    if temp['f1-score'] > max_f1_score and temp != '_t_pad_' : 
                        max_f1_score = temp['f1-score'] 
                        which_max_f1 = item
                        support_max_f1 = temp['support']
                    if temp['f1-score'] < min_f1_score : 
                        min_f1_score = temp['f1-score']
                        which_min_f1 = item 
                        support_max_f1 = temp['support'] 


                d = {
                    'model' : [m] , 
                    'epoch' : [e]  , 
                    "batch_size" : [b] , 
                    'max_f1_score' : [max_f1_score] , 
                    'which_maxf1' : [which_max_f1] , 
                    'support_maxf1' : [support_max_f1] , 
                    'min_f1_score' : [min_f1_score] , 
                    'which_minf1' : [which_min_f1] , 
                    'support_minf1' : [support_max_f1] ,
                    'macro_avg_f1' : [train_report['f1-score']],  
                    'train_acc' : [train_acc] , 
                    'val_acc' : [val_acc] 
                }
                tempdf = pd.DataFrame(data =d )
                df = df.append(tempdf , ignore_index= True )
                print(df )
                print( '============================================================================================================')
# test_y = model.predict(test_x) 
# report = model.evaluate(test_x , real_test_y ) 

Empty DataFrame
Columns: [model, epoch, batch_size, max_f1_score, which_maxf1, support_maxf1, min_f1_score, which_minf1, support_minf1, macro_avg_f1train_acc, val_acc]
Index: []
2020-11-26 14:17:41,070 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 130
2020-11-26 14:17:42,493 [DEBUG] kashgari - predict seq_length: None, input: (23600, 130)
738/738 [==============================] - 37s 50ms/step
2020-11-26 14:18:25,483 [DEBUG] kashgari - predict output: (23600, 130)
2020-11-26 14:18:25,485 [DEBUG] kashgari - predict output argmax: [[ 0  1  1 ...  1  4  1]
 [ 0  1  1 ...  4  4  1]
 [ 0  1  1 ...  2  2  2]
 ...
 [ 0  1  1 ...  1  1  1]
 [ 0  9  1 ...  1 20  1]
 [ 0  1  1 ...  1  4  1]]
2020-11-26 14:18:25,647 [DEBUG] kashgari - predict seq_length: None, input: (2950, 130)
93/93 [==============================] - 5s 51ms/step
2020-11-26 14:18:30,910 [DEBUG] kashgari - predict output: (2950, 130)
2020-11-26 14:18:30,912 [DEBUG] kashgari - pre

KeyboardInterrupt: 

In [ ]:
# print(report )
# get the test pred by loading the models 
# find best f1 score 
# find best epoch 
df

# Output format

In this project, you should predict the NER tags for the test set tokens.

The index of test set starts from 0 and ends with 2949.

You should write the predictions into a .csv file, where the first column is the test indexes in ascending order, and the second column is a json format prediction list.

E.g.

|id|labels|
|:--:|:--:|
|0|`['O', 'O', 'CHEMICAL', 'VIRUS', ...]`|
|1|`['O', 'O', 'GENE_OR_GENOME', ...]`|
|...|...|

Format requirements:
1. The first column `id` should be an integer, in ascending order, starting from 0 and corresponding to the index in test_dict.
2. The second column `labels` should be a dumped string using json, storing the your predictions for each token. The size of the list should be exactly 128, including padding tokens.

### For example, this is your prediction for the test set:

In [ ]:
test_preds_numerical = np.random.randint(1, len(tag_dict), 
                                         (len(test_dict["id"]), max_sent_length))
test_preds = np.array([[idx2tag[p] for p in preds] for preds in test_preds_numerical])
print(test_preds.shape)
print(test_preds[0])

# use the model to make the preds on test and create a matirx called test_preds_numerical
# change the index back to tag 

In [ ]:
# Let's take the baseline 1 as an example, where we predict all labels as 1.

import json
import pandas as pd

df = pd.DataFrame({'id': test_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in test_preds]})
df.to_csv('test_preds.csv', index=False)

In [ ]:
pd.read_csv("test_preds.csv")

# Please make your output-format exactly the same as above

You could check it by playing around with the validation set with our evaluation codes `evaluate.py`:

In [ ]:
# val_preds_numerical = np.random.randint(1, len(tag_dict), 
#                                          (len(val_dict["id"]), max_sent_length))
val_preds = np.array([[idx2tag[p] for p in preds] for preds in np.ones((len(val_dict["id"]), max_sent_length))])

import json
import pandas as pd

df = pd.DataFrame({'id': val_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in val_preds]})
df.to_csv('val_preds.csv', index=False)

from evaluate import evaluate

print("val accuracy", evaluate('val_preds.csv', "data/val.pkl"))